# Oracle 전용
> 국민대학교 김남규 교수 강의:
>
> https://www.youtube.com/playlist?list=PLg_wJlcMiuKtGdlIaAZ0rOPPQuTDENnEQ
## 1. 실습 환경 (택 1)
- Local 환경
    - Oracle XE와 SQL Developer 설치
    - 속도가 빠르며 Query 길이에 제한이 없음
    - 설치가 복잡함
- Cloud 환경
    - https://livesql.oracle.com
    - 설치가 필요 없으며 어디에서나 접속 가능
    - 속도가 느리며 Query 길이에 제한이 있음

DUAL
- 시스템이 제공해 주는 보이지 않는 테이블
- DUMMY라는 컬럼명 한 개에 'X'라는 데이터 하나가 들어있다.
- 예) SELECT 'ABC', 5+4
      FROM DUAL;
    - 출력: 
        ```
           +---+------+-----+
           |   |'ABC' | 5+4 |
           +---+------+-----+
           | 1 | ABC  | 9   |
           +---+------+-----+
        ```

NULL 값
- ORACLE에서 NULL은 가장 큰 값으로 취급 됨 
    - MySQL, Microsoft의 sql server는 반대로 NULL을 가장 작은 값으로 취급
    - -> ORDER BY 컬럼명 ASC 했을 때 NULL값들은 가장 아래에 출력됨

사용자 생성 객체 확인
-   ```mysql
    SELECT * 
    FROM USER_OBJECTS    # USER_OBJECTS는 시스템이 저절로 만들어준 테이블
    WHERE OBJECT_TYPE = 'TABLE';
    ```
    -> 생성된 모든 테이블 출력

타입 (data type)
- 테이블의 타입 확인:
    - DESCRIBE 테이블명
    - 또는 DESC 테이블명
- 문자 CHAR(숫자), VARCHAR2(숫자)
    - CHAR(숫자)
        - 고정 문자열 (기본 1 Byte ~ 최대 2000 Byte)
        - 변수에 할당된 값이 고정 길이보다 작은 경우, 나머지 공간에 공백이 채워짐
        - 'AA' = 'AA '
    - VARCHAR2(숫자)
        - 가변 길이 문자열 (기본 1 Byte ~ 최대 4000 Byte)
        - 변수에 할당된 값이 고정 길이보다 작은 경우, 나머지 공간은 사용하지 않음
        - 'AA' <> 'AA '
- 숫자 NUMBER[(숫자[, 숫자])]
    - 정수, 실수 등의 숫자 정보
    - MS SQL Server의 경우 10가지 이상의 숫자 타입 지원
    - 전체 자리수와 소수 부분 자리수를 명시함
    - 예) NUMBER(8, 2) -> 정수 부분 6자리, 소수 부분 2자리
- 날짜 DATE
    - 날짜와 시간 정보
제약조건
- 종류:
    - PRIMARY KEY (기본키): 기본키 정의 (테이블 당 하나의 기본키 제약만 정의 가능); (NOT NULL, UNIQUE가 자동으로 같이 설정 됨)
        - 컬럼이 2개 이상 묶여 있는 하나의 PRIMARY KEY 설정 가능
        - 예) CREATE TABLE schedule (
            stadium_id             CHAR(3) NOT NULL,
            sche_date              CHAR(8) NOT NULL,
            home_score             NUMBER(2),
            away_score             NUMBER(2),
            CONSTRAINT             schedule_pk PRIMARY KEY (stadium_id, sche_date),
            CONSTRAINT             schedule_fk PRIMARY KEY (stadium_id), REFERENCES stadium (stadium_id)
            );
    - FOREIGN KEY (외래키): 다른 테이블의 기본키를 참조하는 외래키를 지정; 외래키 지정시 참조 무결성 제약 옵션 선택
    - NOT NULL: NULL 값 입력 불허용
    - UNIQUE: 해당 컬럼의 값이 테이블 내에서 유일해야 함을 제약
    - CHECK: 입력할 수 있는 값을 제한
        - 예) CONSTRAINT back_no_ck CHECK (back_no < 99 AND back_no > 10)
- 제약 조건 지정 3가지 방식 (테이블 생성시)
    - 제약 조건명을 명시적으로 부여 가능, 묵시적으로 제약 조건명 없이 제약 조건 설정 가능 (이럴 경우는 시스템에서 알아서 제약조건명을 부여해 주는 것)
    1. 묵시적 방식 (CONSTRAINT의 이름을 지정해주지 않고 컴퓨터가 알아서 지정해줌) - 가장 간단
        - 예) CREATE TABLE player1 (
                        player_id CHAR(7) PRIMARY KEY,
                        player_name VARCHAR(20) NOT NULL,
                        nickname VARCHAR(30) UNIQUE,
                        height NUMBER(3) CHECK (height >= 150 AND height <= 200),
                        team_id CHAR(3) REFERENCES team(team_id)
                );
    2. 명시적 방식 1 (CONSTRAINT의 이름을 지정해 줌)
        - 예) CREATE TABLE player2 (
                        player_id CHAR(7) CONSTRAINT p2_pk_id PRIMARY KEY,
                        player_name VARCHAR2(20) CONSTRAINT p2_nn_name NOT NULL,
                        nickname VARCHAR2(30) CONSTRAINT p2_un_nick UNIQUE,
                        height NUMBER(3) CONSTRAINT p2_ck_height CHECK (height >= 150 AND height <= 200),
                        team_id CHAR(3) CONSTRAINT p2_fk_tid REFERENCES team(team_id)
                );
    3. 명시적 방식 2
        - 예) CREATE TABLE player3 (
                        player_id CHAR(7),
                        player_name VARCHAR2(20) CONSTRAINT p3_nn_name NOT NULL,
                        nickname VARCHAR2(30),
                        height NUMBER(3),
                        team_id CHAR(3),

                        CONSTRAINT p3_pk_id PRIMARY KEY (player_id),
                        CONSTRAINT p3_un_nk UNIQUE (nickname),
                        CONSTRAINT p3_ck_height CHECK (height >= 150 AND height <= 200),
                        CONSTRAINT p3_fk_tid FOREIGN KEY (team_id) REFERENCES team(team_id)
                );
- 제약 조건 확인
    - SELECT *
        FROM ALL_CONSTRAINTS          # ALL_CONSTRAINTS는 시스템이 저절로 만들어준 테이블
        WHERE table_name IN ('player1', 'player2', 'player3')
        ORDER BY CONSTRAINT_NAME;
- FK 제약 조건 옵션 설정
    - 예) CONSTRAINT fk1 FOREIGN KEY (team_id) REFERENCES team(team_id)
            ON DELETE CASCADE ON UPDATE RESTRICT;
    - Referential Triggered Action
        - ON UPDATE -> 참조되는 키가 UPDATE 될 때 할 Action 설정
        - ON DELETE -> 참조되는 키가 DELETE 될 때 할 Action 설정
    - Referential Action (뭘 할 것인지)
        - RESTRICT (Default): 기본값의 삭제/갱신을 불허 -> 가장 대표적인 방법
        - NO ACTION: RESTRICT와 동일하게 동작
        - CASCADE:
            - 기본키가 삭제되면 해당 값을 외래키로 갖는 레코드(모든 행)도 삭제
            - 기본키가 갱신되면 이를 참조하는 외래키를 새로운 값으로 갱신
        - SET NULL:
            - 기본키가 삭제/갱신되면 이를 참조하는 외래키를 NULL로 갱신
- 기존 테이블을 활용한 테이블 생성 (SELECT문 활용)
    - CREATE TABLE player_temp AS SELECT * FROM player;
        - player의 모든 레코드로 player_temp라는 테이블을 만드는 것
    - 참조하여 테이블을 만들 때 제약조건은 NOT NULL만 복제됨
        - PK, FK, UNIQUE, CHECK 등은 수동(직접)으로 추가해야 됨

ORACLE 연산자
- 연산자 우선순위
    1. 괄호 ()
    2. NOT 연산자
    3. 비교 연산자, SQL 연산자
    4. AND
    5. OR
- 산술 연산자
    - +, -, *, /, %(나머지)
    - number와 date 자료형에 적용 (MySQL에서는 날짜 시간에는 바로 적용 안 됨 )
    - date에 적용 시 날짜 값으로 적용됨 (23/01/29인 값에 +3 하면 23/02/01이 출력됨)
    - Null 값에 산술 연산의 결과는 항상 NULL
- 비교 연산자
    - >, <, >=, <=, =
    - 모든 자료형에 적용 (문자열 크기는 사전순 - 먼저 나오는 것이 작음)
        - 예) '01' < '02' < '1' < '11' < '2'  
    - !=(같지 않음), <>(같지 않음), 
    - !< (왼쪽 값이 오른쪽 값보다 작지 않음)
    - !> (왼쪽 값이 오른쪽 값보다 크지 않음)
    - NULL에는 비교 연산자 사용 불가
- 논리 연산자
    - 모든 자료형에 적용
    - NOT, AND, OR
    - 진리표
        - AND (*와 같은 개념)
            - 참 AND 참 = 참
            - 참 AND 거짓 = 거짓
            - 거짓 AND 참 = 거짓
            - 거짓 AND 거짓 = 거짓
            - 참 AND NULL = NULL
            - 거짓 AND NULL = 거짓
            - NULL AND NULL = NULL
        - OR (+와 같은 개념)
            - 참 OR 참 = 참
            - 참 OR 거짓 = 참
            - 거짓 OR 참 = 참
            - 거짓 OR 거짓 = 거짓
            - 참 OR NULL = 참
            - 거짓 OR NULL = NULL
            - NULL OR NULL = NULL
        - NOT
            - 참 = NOT 거짓
            - 거짓 = NOT 참
            - NULL = NOT NULL
- SQL 연산자
    - 합성(연결) 연산자 - 문자열 결합  
        - || (파이프): str1 || str2
            - 2개 이상 가능: str1 || str2 || str3 || str4 ...
                - 작동하는 방법: (((str1 || str2) || str3) || str4) ... (이항?)
            - 예) str = 'a' || 'bcd'
            >> str = 'abcd'
        - CONCAT(str1, str2)
            - 2개만 연결 - 3개 이상 불가능
    - BETWEEN + AND, NOT BETWEEN + AND
        - 예) SELECT name FROM author WHERE id BETWEEN 1 AND 3
    - IN (list), NOT IN (list)
        - 예) SELECT name FROM author WHERE id IN (1, 2, 3)
        - 예) SELECT name AS "작가 이름"
                FROM author
                WHERE (id, name) IN ((1, '김%'), (2, '이%'));
    - IS NULL, IS NOT NULL
        - 주의: id = NUlL 또는 id <> NULL 할 시 에러가 난다 (이유: NULL은 비교 연산자 사용 불가)
    - LIKE
        - 문자열 비교 연산
        - 와일드카드 사용 가능 
            - '%' 임의의 문자 N개 / '_' 임의의 문자 1개 ('__'는 임의의 문자 2개, '___'는 임의의 문자 3개, and so on)

DDL (Data Definition Lang) 데이터 정의어: 테이블/컬럼 정의/변경하는 명령어
- CREATE TABLE 테이블명 (필드이름1 필드타입1, 필드이름2 필드타입2, ...)
    - 테이블 생성
        - 다른 테이블의 값을 참조할 수 있음 (REFERENCES)
        - 예) CREATE TABLE stadium (
                        stadium_id           CHAR(3) NOT NULL,
                        stadium_name         VARCHAR2(40) NOT NULL,
                        hometeam_id          CHAR(3),
                        seat_count           NUMBER,
                        address              VARCHAR2(60),
                        tel                  VARCHAR2(10),
                        CONSTRAINT           stadium_pk PRIMARY KEY (stadium_id)
                );
        - 예) CREATE TABLE team (
                        team_id              CHAR(3) NOT NULL,
                        region_name          VARCHAR2(8) NOT NULL,
                        team_name            VARCHAR2(40) NOT NULL,
                        stadium_id           CHAR(3) NOT NULL,
                        address              VARCHAR2(60),
                        tel                  VARCHAR2(10),
                        CONSTRAINT           team_pk PRIMARY KEY (team_id),
                        CONSTRAINT           team_fk FOREIGN KEY (stadium_id) REFERENCES stadium (stadium_id)
                );
            - FOREIGN KEY 설정 시 해당 키의 값을 포함하고 있는 테이블이 먼저 생성되어 있어야 함 -> stadium_id를 갖고 있는 stadium이라는 테이블이 먼저 생성되어 있어야 함
    - 생성 규칙:
        - 테이블명: 단수형 권장, 다른 테이블과 중복 불가
        - 컬럼명: 테이블 내 중복 불가, 생성시 각 컬럼들은 괄호 내에서 콤마로 구분, 칼럼 뒤에 데이터 유형 지정 필수
        - 제약조건명(constraints): 다른 제약조건의 이름과 중복 불가 (제약: PRIMARY KEY, UNIQUE KEY, NOT NULL, CHECK, FOREIGN KEY 등)
        - 예약어(reserved word; 예, SELECT, FROM, WHERE, UPDATE, SET 등) 사용 불가
        - 문자, 숫자, 일부 기호 (_, $, #)만 허용
        - 반드시 문자로 시작 가능 (숫자, 기호 불가)
    - 기존 테이블을 활용한 테이블 생성 (SELECT문 활용)
        - CREATE TABLE player_temp AS SELECT * FROM player;
            - player의 모든 레코드로 player_temp라는 테이블을 만드는 것
        - 참조하여 테이블을 만들 때 제약조건은 NOT NULL만 복제됨
            - PK, FK, UNIQUE, CHECK 등은 수동(직접)으로 추가해야 됨
- RENAME문
    - 테이블명 변경
        - RENAME 원래테이블명 TO 새로운테이블명;
        - 예) RENAME author TO author_new;
- ALTER TABLE 테이블명
    - 수정 (TABLE 구성변경)
    - 컬럼 추가
        - ALTER TABLE 테이블명 ADD (새로운필드명 필드타입);
        - 예) ALTER TABLE player_temp ADD (address VARCHAR2(80));
        - ** MySQL에서는 ADD 뒤에 COLUMN도 들어감
            - MySQL: ALTER TABLE 테이블명 ADD COLUMN 새로운필드명 필드타입;
                - 예) ALTER TABLE author ADD COLUMN updated_at DATETIME;
    - 컬럼 정의 수정 
        - ALTER TABLE 테이블명 MODIFY ~
        - 허용되지 않는 경우: 이미 입력되어 있는 값에 영향을 미치는 변경은 불가능
        - 허용되는 경우:
            - 데이터 타입 변경 -> 테이블에 아무 행도 없거나, 해당 컬럼이 NULL만 갖고 있을 때 가능
            - 컬럼의 크기 변경
                - 컬럼의 크기 확대 -> 항상 가능
                - 컬럼의 크기 축소 -> 테이블에 아무 행도 없거나, 
                                    해당 컬럼이 NULL만 갖고 있거나,
                                    현재 저장된 값을 수용할 수 있는 크기로의 축소만 가능
            - DEFAULT 값 추가/수정 -> 추가/수정 이후 삽입되는 행에만 영향을 미침
            - 제약조건
                - NOT NULL
                    - NOT NULL 제약조건 추가 -> 테이블에 아무 행도 없거나, 해당 컬럼에 NULL이 없을 때 가능
                        - ALTER TABLE 테이블명 MODIFY (컬럼명 [NOT] NULL)
                    - NOT NULL 제약조건 삭제 -> 항상 가능
                - 다른 제약 조건들 -> 테이블 생성 후에도 제약조건 추가/삭제 가능
                    - 제약조건 확인
                        - SELECT * FROM ALL_CONSTRAINTS
                            WHERE table_name = 'player_temp';
                    - 제약조건 추가
                        - ALTER TABLE 테이블명 ADD CONSTRAINT 제약조건명 ~
                        - 예) player_temp 테이블에 team에 대한 FOREIGN KEY 추가
                            - ALTER TABLE player_temp
                                ADD CONSTRAINT player_temp_fk FOREIGN KEY (team_id) REFERENCES team (team_id);
                    - 제약조건 삭제
                        - ALTER TABLE 테이블명 DROP CONSTRAINT 제약조건명
                        - 예) player_temp 테이블에 team에 대한 FOREIGN KEY 
                            - ALTER TABLE player_temp
                                DROP CONSTRAINT player_temp_fk;
    - 컬럼 삭제
        - ALTER TABLE 테이블명 DROP COLUMN 삭제할컬럼명;
    - 제약 추가
        - ALTER TABLE 테이블명 ADD CONSTRAINT 제약명 제약조건(컬럼명);
        - 예) ALTER TABLE test_tab ADD CONSTRAINT const1 UNIQUE(col1);
        - 테이블(test_tab)의 컬럼(col1)에 const1이라는 제약명으로 UNIQUE 제약 설정
    - 제약 삭제
        - ALTER TABLE 테이블명 DROP CONSTRAINT 제약명;
        - 예) ALTER TABLE test_tab DROP CONSTRAINT const1;
        - 테이블(test_tab)에 설정되어있는 const1이라는 제약명을 가진 제약을 삭제한다
    - 제약 활성화/비활성화
        - ALTER TABLE 테이블명 [ENABLE/DISABLE] CONSTRAINT 제약명;
        - 예) ALTER TABLE test_tab DISABLE CONSTRAINT const1;
        - 테이블(test_tab)에 설정된 const1라는 제약명을 가진 제약을 일시적으로 비활성화한다
- DROP TABLE 테이블명
    - 삭제
        - 예) DROP TABLE team;
        - 그러나 다른 테이블이 team 테이블의 값을 참조하고 있을 경우 에러 발생할 것임
            - 예) player 테이블의 player_id 컬럼이 team 테이블의 player_id 컬럼을 foreign key로 참조하고 있을 경우
    - 관련된 제약조건을 함께 삭제
        - 예) DROP TABLE 테이블명 CASCADE CONSTRAINT;

DML (Data Manipulation Lang) 데이터 조작어: 데이터 조회/추가/수정/삭제
- SELECT 
    - 조회 - 조건이 없으면 모든 data 조회
    - 문법: SELECT [ALL/DISTINCT] 컬럼/필드명 FROM 테이블명 WHERE 조건
    - 컬럼/필드명을 명시해도 되고, * 을 통해 모든 컬럼 조회도 가능
    - * 
        - 테이블의 모든 데이터 출력
        - 예) SELECT * 
                FROM author;
    - ALL
        - 중복 데이터 모두 출력 (default)
        - 문법: SELECT [ALL] 컬럼/필드명 FROM 테이블명
        - 예) SELECT [ALL] name, email 
                FROM author;
    - DISTINCT 
        - 중복되는 값 제거
        - 문법: SELECT DISTINCT 컬럼명1 [, 컬럼명2, 컬럼명3 ...] FROM 테이블명;
        - DISTINCT 키워드는 SELECT 뒤와 첫 컬럼의 앞에 위치해야 함
        - NULL 값도 하나의 값으로 간주함
            - ORACLE에서 NULL값은 가장 큰 값으로 간주됨 -> ORDER BY 했을 때 가장 큰 값으로 간주됨
        - 예) SELECT DISTINCT name
                FROM author;
            - 중복된 name들 제거되고 하나씩만 반환
        - 컬럼이 하나 이상인 경우 모든 컬름들의 조합을 출력
        - 예) SELECT DISTINCT name, email 
                FROM author;
            - name과 email의 가능한 조합 반환
    - ROWNUM
        - TOP N개의 레코드를 반환
            - 상위 N개이므로 RONUM 뒤에는 =, >, >= 사용 불가; 다만, 1인 경우만 = 사용 가능
                - 얘) SELECT 컬럼명
                        FROM 테이블명
                        WHERE ROWNUM = 1; --> 상위 1개 출력
            - 오로지 RONUM <= 숫자, RONUM < 숫자 만 사용 가능
        - 사용자가 아닌 시스템이 관리하는 Pseudo Column (i.e., 쓸 수는 있지만 진짜 컬럼은 아닌 컬럼)
        - 채번, 출력 개수 지정 등에 활용 가능
        - 예)SELECT 컬럼명
                FROM 테이블명
                WHERE ROWNUM <= 숫자;
        - 주의: 다른 database system들은 다른 키워드를 사용한다
            - SQL Server / MS Access:
                - 문법: SELECT TOP 숫자 컬럼명
                        FROM 테이블명;
                - percent도 가능: SELECT TOP 숫자 PERCENT 컬럼명
                                    FROM 테이블명;
            - MySQL:
                - 믄밥: SELECT 컬럼명
                        FROM 테이블명
                        WHERE 조건
                        LIMIT 숫자;
            - Oracle 12:
                - 문법: SELECT 컬럼명
                        FROM 테이블명
                        ORDER BY 컬럼명
                        FETCH FIRST 숫자 ROWS ONLY;
                - percent도 가능: SELECT 컬럼명
                                    FROM 테이블명
                                    ORDER BY 컬럼명
                                    FETCH FIRST 숫자 PERCENT ROWS ONLY;
            - Older Oracle:
                - 문법: SELECT 컬럼명
                        FROM 테이블명
                        WHERE ROWNUM <= 숫자;
                - 문법 (with ORDER BY): SELECT 컬럼명
                                        FROM (SELECT 컬럼명 FROM 테이블명 ORDER BY 컬럼명)
                                        WHERE ROWNUM <= 숫자;
    - WHERE ~
        - 조건 걸기, 특정 조건을 만족하는 데이터 한정 출력
        - 문법: SELECT 컬럼명 FROM 테이블명 WHERE 조건
            - WHERE 컬럼명 (연산자) 컬럼값
        - 예) ... WHERE 컬럼명 = 컬럼값
    - ORDER BY
        - 선택한 결과의 정렬 (항상 조건 다 걸고 마지막에 붙임)
        - 문법: ORDER BY 컬럼명 [ASC/DESC]
        - SELECT 문으로 선택한 결과를 ORDER BY 절을 사용하여 정렬
        - 기본 설정은 ASC
        - 여러 컬럼의 데이터를 쉼표(,)를 사용하여 한 번에 정렬 가능
        - 예) SELECT * 
                FROM author 
                ORDER BY id DESC;
            - id를 기준으로 내림차순 정렬
        - 예) SELECT * 
                FROM author 
                ORDER BY name [ASC];
            - name을 기준으로 오름차순 정렬 [ASC]는 optional
        - 예) SELECT name, email
                FROM author
                ORDER BY 2;
            - 컬럼이 하나 이상일 경우 처음 컬럼부터 번호가 매겨진다 (name = 1, email = 2, ...)
            - ORDER BY 2에서 '2'는 SELECT 뒤 두번째로 명시된 컬럼인 'email'을 가리킨다
            - 그러므로 email을 기준으로 오름차순 정렬
        - ORDER BY절 생략시 PK를 기준으로 정렬하여 결과값 반환
        - ORDER BY절에는 SELECT절에 없는 컬럼도 사용 가능
        - 예) SELECT name 
                FROM author 
                ORDER BY email;
            - email을 기준으로 author 테이블의 name들이 오름차순 정렬로 출력
        - 예제) name만을 기준으로 오름차순 정렬
            - SELECT * FROM author ORDER BY name ASC;
        - 예제) name, email 기준으로 오름차순 정렬; name 중복일경우, email을 기준으로 정렬하게 되는 것
            - SELECT * FROM author ORDER BY name, email;
        - 예제) name은 오름차순, email은 내림차순으로 정렬
            - SELECT * FROM author ORDER BY name ASC, email DESC;
    - GROUP BY
        - 선택된 레코드의 집합을 툭정조건으로 그룹화한 결과 집합
        - 데이터의 값을 집계, 주로 집계 함수와 같이 사용
        - 집계 함수 예) SUM(), COUNT(), AVG(), MIN(), MAX() 등
        - 예) 작가가 쓴 글의 갯수를 알아보기; 이름, 이메일, 글 쓴 갯수 출력
            - SELECT a.name AS author_name, a.email, 
                        COUNT(p.author_id) AS 'post_count' 
                FROM author a 
                LEFT JOIN post p ON a.id = p.author_id 
                GROUP BY a.email;
        - 예) sales테이블에서 지점별(지점ID) 평균매출 구하기
            - SELECT branch_id, AVG(amount) AS average 
                FROM sales 
                GROUP BY branch_id;
    - GROUP BY + HAVING 조건절
        - 구문 전체에 대한 조건절은 WHERE ~ 이고 그룹화된 데이터에 대한 조건절은 HAVING ~
        - 예) author에서 중복되는 이름과 중복되는 횟수 조회
            - SELECT name, COUNT(*) AS COUNT 
                FROM author 
                GROUP BY name 
                HAVING COUNT(*) > 1;
        - 예) 2022년 한해동안 지점별 총매출을 구하라. 정렬은 금액 내림차순. 총매출이 2000 이상인 지점만 출력. 출력 - 지점명, 총매출액
            - SELECT b.name, SUM(s.amount)
                FROM branches AS b 
                INNER JOIN sales AS s 
                ON b.id = s.branch_id 
                WHERE s.created_date >= '2022-01-01' 
                AND s.created_date < '2023-01-01' 
                GROUP BY b.name 
                HAVING SUM(s.amount) >= 2000.00;
    - AS
        - 테이블과 필드에 임시로 별칭(alias)을 부여하고, 해당 별칭을 SELECT문에서 사용
        - AS 라는 키워드를 줘서 별칭(alias)를 이용한 처리 
        - 문법: SELECT 컬럼명 [AS] 별칭 FROM 테이블이름;
        - 생략 가능
        - 예) SELECT COUNT(DISTINCT name) AS count_name 
                FROM author;
        - 예) SELECT COUNT(DISTINCT name) count_name 
                FROM author;
        - 출력되는 별칭은 WHERE, FROM, GROUP BY, HAVING, ORDER BY 뒤에는 사용하지 않는다; 임시로 부여한 것이기 때문에 명령어 안에서는 사용되지 않는다
        - 별칭이 공백, 특수문자 등 포함 시 큰 따옴표("") 사용
            - 공백이나 특수문자 포함 안해도 "" 사용해도 됨
        - 예) SELECT name AS "작가 이름",
                        date AS "날짜!"
                        content AS "내용"
                FROM author
- INSERT INTO 
    - 테이블 한 건의 레코드 추가
        - 문법1: INSERT INTO 테이블명 (필드명1, 필드명2, 필드명3 ...) VALUES (데이터값1, 데이터값2, 데이터값3 ...);  -> 필드타입 넣을 필요 없음
            - 예) INSERT INTO author(id, name, email) values (1, 'leo', 'leo@gmail.com');
            - 컬럼 순서는 실제 테이블의 컬럼 순서와 무관
            - 정의하지 않은 컬럼은 NULL이 입력됨
        - 문법2: INSERT INTO 테이블명 VALUES (전체 컬럼의 value_list)
            - 예) INSERT INTO Pearl VALUES (13, 'merona', 'binggrae', 'melon', 600)
            - 전체 컬럼의 모든 값을 순서대로 입력해야 됨
            - 빈 값은 NULL 로 입력
                - 주의: MySQL에서는 안 되지만 ORACLE에서는 NULL 또는 작은 따옴표 ('')로 입력 가능 (**주의: 작은 따옴표 안에 공백이 있으면 안 됨 - 예) ' '는 하나의 공백 값으로 인식)
    - 여러 건 입력 시 INSERT ALL ~
        - 언제 사용?
            - 테이블 생성 후 초기 데이터 일괄 업로드
                - 문법: INSERT ALL
                        INTO 테이블명 VALUES (전체 컬럼의 value_list)
                        INTO 테이블명 VALUES (전체 컬럼의 value_list)
                        INTO 테이블명 VALUES (전체 컬럼의 value_list)
                        SELECT * FROM DUAL;
            - 기존 테이블의 레코드 조회 후 다른 테이블에 삽입 (예, 회원 명단 조회해서 그 중 미납 회원을 뽑아서 별도의 미납자 테이블 생성)
                - 문법: INSERT ALL
                                INTO 테이블2 VALUES (컬럼1, 컬럼2, ...)
                                INTO 테이블3 VALUES (컬럼1, 컬럼3, ...)
                        SELECT 컬럼1, 컬럼2, 컬럼3 FROM 테이블1
                    - 테이블1에서 레코드를 반환하여 테이블2, 테이블3에 분할 저장
        - 반드시 SELECT 구문도 함께 사용해야 됨 (SELECT 없을 시 에러 남)
    - 문자 또는 날짜 값은 작은 따옴표로 묶음 (숫자 데이터는 작은 따옴표 없이 사용)
- UPDATE 
    - 데이터 수정
    - 문법: UPDATE 테이블명 SET 필드명1 = 수정값1, 필드명2 = 수정값2 [WHERE 필드명 = 데이터값];
    - 조건이 없으면 다 바꿈 (조심해야 함)
    - 예) UPDATE author 
            SET email = 'pearl.ahn@gmail.com' 
            WHERE id = '2';
        - author 테이블의 id가 '2'인 레코드의 email을 'pearl.ahn@gmail.com'으로 바꿈
    - 예) UPDATE author
            SET email IS NULL
        - author 테이블의 모든 레코드의 email을 NULL값으로 바꿈
    - UPDATE와 ROWNUM을 이용한 채번
        - 테이블 내의 UNIQUE한 일련번호를 생성하는 경우
        - 1단계: ROWNUM을 담을 새 컬럼을 만든다
            - 예) ALTER TABLE 테이블명 ADD (새컬럽명 NUMBER);
                    ALTER TABLE Pearl ADD (ROW_ID NUMBER)
                - 'Pearl'이라는 테이블에 'ROW_ID'의 이름을 가지고 NUMBER라는 타입을 가진 빈 컬럼 추가
        - 2단계: 해당 컬럼에 ROWNUM값을 준다
            - 예) UPDATE 테이블명 SET 새컬럼명 = ROWNUM;
                    UPDATE Pearl SET ROW_ID = ROWNUM;
                - 'ROW_ID'라는 컬럼에 해당 레코드의 ROWNUM 값으로 수정
- DELETE
    - 데이터 삭제 (테이블에 존재하는 전체 레코드 삭제)
    - 문법: DELETE [FROM] 테이블명 [WHERE 필드명 = 데이터값];
        - DELETE 테이블명; 또는 DELETE FROM 테이블명
        - 주의: MySQL에서는 꼭 FROM을 써줘야 된다 -> DELETE FROM ~
    - 한 row 전체를 지우기 때문에 컬럼명이나 *를 지정해줄 필요 없다
    - 삭제 후 복구 가능 (TRUNCATE는 복구 불가)
    - 조건을 빼버리면 테이블 전체를 지움 (조심해야 됨)
    - 예) DELETE FROM author 
            WHERE id >= 10;

DCL (Data Control Lang) 데이터 제어어: 사용자 권한 관리 (계정 생성 후 권한부여)
- 사용자의 계정 생성/삭제, 권한 부여/회수
- GRANT, REVOKE, CREATE USER, ALTER USER, DROP USER
    - GRANT (권한 부여): GRANT CREATE USER TO mis1;
    - REVOKE (권한 취소): REVOKE CREATE USER FROM mis1;
- 계정 (USER) 관련
    - 계정 생성: CREATE USER 계정명 IDENTIFIED BY 비밀번호;
    - 계정 비밀번호 변경: ALTER USER 계정명 IDENTIFIED BY 새비밀번호;
    - 유저 생성 권한 부여: GRANT CREATE USER TO 계정명;
    - 유저 생성 권한 회수: REVOKE CREATE USER FROM 계정명;
    - 계정 삭제: DROP USER 계정명 CASCADE;
        - DROP 연산 수행시:
            - CASCADE 옵션 사용시 사용자가 생성한 객체도 함께 삭제됨
            - CASCADE 옵션 미사용시 사용자가 객체를 갖고 있지 않은 경우에만 삭제 실행
- SESSION 관련
    - 로그인을 위해 CREATE SESSION 권한 필요: GRANT CREATE SESSION TO 계정명;
- Object 권한 관련
    - 테이블 생성을 위해 CREATE TABLE 권한 필요: GRANT CREATE TABLE TO 계정명;
    - 다른 Object 권한
        - GRANT 권한 ON 소유계정.테이블명 TO 계정명
        - 계정1 계정에 PLAYER라는 테이블이 있는 경우:
            - 관리자 계정으로 접속한 상태시: GRANT SELECT ON 어느계정1.PLAYER TO 계정2;
            - 계정1 계정으로 접속한 상태시: GRANT SELECT ON PLAYER TO 계정2;
            - 계정2 계정에서 계정1 계정의 PLAYER 테이블 조회: SELECT * FROM 계정1.PLAYER;
            - 계정1 계정에서 자신 계정의 PLAYER 테이블 조회: SELECT * FROM PLAYER;
    - 객체별 권한의 종류

        <img src="images/image3.png" width="30%">

- ROLE을 이용한 권한 부여
    - ROLE: 권한들의 Package

        <img src="images/image4.png" width="30%">
    - 예)
        ```sql
        CREATE ROLE MY_ROLE
        GRANT       CREATE SESSION, CREATE TABLE TO MY_ROLE;
        GRANT       MY_ROLE TO 계정1;
        ```
- Oracle의 ROLE
    - 일반적으로 사용자 생성시 CONNECT, RESOURCE를 통해 기본 권한을 부여한다 (DBA라는 ROLE과 다른 ROLE들도 있음)
    - 각 권한과 권한들을 모아놓은 각 ROLE을 같이 사용할 수 있다 (섞어서 새로운 ROLE 생성 또는 어느 유저에게 권한 부여 가능)
    - 계정 생성 코드 (환경 설정시)
        ```sql
        CREATE USER myid IDENTIFIED BY mypw;
        GRANT CONNECT, DBA, RESOURCE TO myid;
        ```
    - ROLE의 권한 확인
        ```sql
        SELECT      GRANTEE, PRIVILEGE
        FROM        DBA_SYS_PRIVS
        WHERE       GRANTEE IN('CONNECT', 'RESOURCE')
        ORDER BY    GRANTEE;
        ```
        - 출력값:
            <img src="images/image5.png" width="10%">
- 아래 왼쪽은 system이라는 아이디와 admin이라는 비밀번호로 들어감, 오른쪽은 mis1이라는 아이디와 new_pw라는 비밀번호로 들어감

    <img src="images/image2.png" width="30%">


TCL (Transaction Control Lan) 트랜젝션 제어어: 변경 내용을 확정/취소; COMMIT, ROLLBACK
- DB의 논리적 연산 단위
    - 의미적으로 분할할 수 없는 최소의 단위
    - 일반적으로 하나의 트랜잭션은 여러 SQL 문장을 포함함
    - 성공시 모든 연산을 반영, 취소시 모든 연산을 취소함 -> All or Nothing
- 사용 예)
    - 도서 주문: 재고 수량 감소, 주문 내역 생성, 결제, 포인트 적립
    - 계좌 이체: 원 계좌의 잔액 감소, 다른 계좌의 잔액 증가
    - 교통카드 충전: 잔액 증가, 결제
- 트랜잭션은 SQL문 실행시 자동시작, COMMIT/ROLLBACK 실행시 종료
- 자동 COMMIT/ 자동 ROLLBACK
    - DDL문장 수행시 DDL 수행 전에 자동으로 COMMIT (auto commit)
    - DB를 정상적으로 접속 종료하면 자동 COMMIT
    - 애플리케이션의 이상 종료로 DB와의 접속이 단절되었을 때 자동 ROLLBACK

- 트랜잭션의 특성 (ACID 특성)
    - 트랜잭션의 ACID 특성을 보장하기 위해 DBMS는 동시성 제어(Concurrency Control)를 수행
        - Lock 기반, Timestamp 기반
        
    |특성|설명|
    |--|--|
    |원자성 Atomicity|트랜잭션에서 정의된 연산들은 모두 성공적으로 실행되던지 전혀 실행되지 않은 상태로 남아 있어야 함 (all or nothing)|
    |일관성 Consistency|트랜잭션이 실행 되기 전의 DB 내용이 잘못 돼있지 않다면, 트랜잭션이 실행된 후에도 DB의 내용에 잘못이 있으면 안된다|
    |고립성 Isolation|트랜잭션이 실행되는 도중에 다른 트랜잭션의 영향을 받으면 안됨|
    |지속성 Durability|트랜잭션이 성공적으로 수행되면 그 트랜잭션이 갱신한 DB의 내용은 영구적으로 저장됨|

- 트랜잭션은 제어하기 위한 명령어: COMMIT, ROLLBACk
- COMMIT
    - 변경된 내용을 DB에 영구적으로 반영
    - COMMIT 실행 전 상태에서는
        ```sql
        UPDATE PLAYER SET HEIGHT = HEIGHT + 10;
        ```
        - 변경된 내용은 메모리에 임시로 저장됨
        - 현재 사용자는 증가한 HEIGHT 값을 읽을 수 있음
        - 다른 사용자는 증가 전 HEIGHT 값만 읽을 수 있음
        - HEIGHT에는 잠금(Locking)이 설정되어 다른 사용자는 값을 변경할 수 없음
    - COMMIT 실행 후
        ```sql
        COMMIT;
        ```
        - 변경된 내용은 DB에 저장됨
        - 변경 내용을 모든 다른 사용자가 볼 수 있음
        - 이전 데이터는 모두 사라짐 (별도 로그 보관시 복구 가능)
        - 관련된 행에 대한 잠금이 해제되어 모든 사용자가 변경할 수 있음
- ROLLBACK
    - 기본 - 변경된 내용을 버리고 변경 전 상테(마지막 COMMIT)로 복귀
    - SAVEPOINT: 부분 복귀를 위해 지정한 저장점
        - SAVEPOINT를 지정한 경우, 지정한 저장점(SAVEPOINT)까지만 복귀(ROLLBACK)
        - 특정 저장점까지 롤백하면 그 이후의 명령과 저장점은 모두 무효가 됨
        - 일부 tool에서는 지원되지 않음
        - 동일 이름으로 여러 저장점 정의시 나중에 정의한 저장점이 유효
            ```sql
            ...
            SAVEPOINT PT1;
            ...
            SAVEPOINT PT1;
            ...
            SAVEPOINT PT2;
            ...
            ROLLBACK TO PT1;
            ```
            - 이 경우 두 번째 PT1으로 ROLLBACK --> 두 번째 PT1 이후는 무효가 됨
    - ROLLBACK 이후의 데이터 상태
        ```sql
        ...
        COMMIT;

        UPDATE PLAYER SET HEIGHT = HEIGHT + 10;
        DELETE FROM PLAYER;

        ROLLBACK;
        ```
        - 변경한 내용이 모두 취소됨
        - 이전 데이터가 다시 재저장 됨
        - 관련된 행에 대한 잠금이 해제되어 모든 사용자가 변경할 수 있음


함수 유형
- 생성 주체
    - 사용자 정의 함수 (User Defined Function) - 사용자가 정의
    - 내장 함수 (Built-in Function) - 벤더가 정의 
        - 예) Oracle에서 SUM(), LENGTH() 등등
- 적용 범위
    - 단일 행 함수 (Single-Row Function)
        - 문자형 함수, 숫자형 함수, 날짜형 함수
        - 제어 함수, 변환 함수, NULL 관련 함수
        - 특징:
            - 각 행(row)에 대해 개별적으로 작용하고 그 결과를 반환
                - 단일 행 내에 있는 하나/복수의 값을 인수로 사용
                - 여러 행에 걸친 값을 사용할 수 없음
            - 함수 중첩 (함수의 인자로 함수를 사용) 가능
            - SELECT, WHERE, ORDER BY 절에 사용 가능
        - 예) SELECT player_name, LENGTH(player_name) AS 길이
                FROM player;
                >> 'player_name'과 '길이'라는 필드이름을 가진 테이블이 출력되고, 각 행에는 player_name과 player_name의 길이가 숫자로 담긴 값들이 담겨져 있을 것
                >> 예) player_name이 '윤희준'이면 길이는 3
        - Oracle의 단일행 내장 함수
            - 문자형 함수: 문자형 변수 처리 
                - 예) LOWER(), UPPER(), ASCII(), CHR(), `CONCAT()`, `SUBSTR()`, `LENGTH()`, `LTRIM()`, `RTRIM()`, `TRIM()` 
                - ASCII() vs CHR()
                    - ASCII()는 ()안의 문자/숫자를 아스키 코드 형태로 바꿔주고, CHR()는 ()안의 숫자를 문자/숫자로 바꿔준다
                        - SELECT ASCII('A') FROM DUAL; >> 65
                        - ACVII('A') vs CHR(53) -> 서로의 값을 반환
                - SUBSTR(문자열, m[, n])
                    - 문자열에서 m번째부터 마지막/n번째 까지 출력
                - LTRIM(문자열[, 지정문자])
                    - 문자열의 왼쪽부터 시작해 다른 문자를 만나기 전까지 지정문자 제거 (지정 문자 생략 시 default = 공백 값 제거)
                    - 예) LTRIM('   xxxYYZZxYZ') -> 'xxxYYZZxYZ'
                    - 예) LTRIM('xxxYYZZxYZx', 'x') -> 'YYYZZxYZx'
                - RTRIM(문자열[, 지정문자])
                    - 문자열 오른쪽부터 시작해 다른 문자를 만나기 전까지 지정문자 제거 (지정 문자 생략 시 default = 공백 값 제거)
                - TRIM(문자열) 또는 TRIM(지정문자 FROM 문자열)
                    - 문자열의 양쪽에서 시작해 다른 문자를 만나기 전까지 지정 문자 제거
                    - 예) TRIM('   xxYYZZ   ') -> 'xxYYZZ'
                    - 예) TRIM('x' FROM 'xxYYZZxYZxx') -> 'YYZZxYZ'
            - 숫자형 함수: 숫자형 변수 처리
                - 예) ABS(), SIGN(), `MOD()`, CEIL(), FLOOR(), `ROUND()`, TRUNC(), SIN(), COS(), TAN(), EXP(), POWER(), SQRT(), LOG(), LN()
                - ABS(숫자)
                    - 절대값 출력
                    - 예) ABS(15) -> 15
                    - 예) ABS(-15) -> 15
                - SIGN(숫자)
                    - 숫자의 양/음/0을 출력
                    - 예) SIGN(-15) -> -1
                    - 예) SIGN(10) -> 1
                    - 예) SIGN(0) -> 0
                - MOD(숫자1, 숫자2)
                    - 나머지 연산 (%); 숫자1/숫자2의 나머지값
                    - 예) MOD(7, 3) -> 1
                    - n개의 조편성을 할 때 자주 사용
                    - 예) PLAYER 테이블에서 PLAYER_ID를 활용하여 전체 선수를 4개의 그룹(0~3)에 배정하시오
                        - PLAYER_ID의 번호값(문자열)이 골고루 퍼져있다는 전제하에,
                        - SELECT PLAYER_ID MOD(PLAYER_ID, 4) FROM PLAYER;
                            - 각 PLAYER_ID마다 0, 1, 2, 3이 부여된 컬럼이 출력될 것임
                            - 그러나 위에서는 암시적 데이터변환이 일어났음 (문자열 PLAYER_ID를 시스템이 자동으로 숫자형으로 변환하여 4로 나눔)
                            - 그러므로 더 성능이 좋게 하는 쿼리는 다음과 같음
                            - SELECT PLAYER_ID MOD(TO_NUMBER(PLAYER_ID), 4) FROM PLAYER;
                            - 이럴경우 시스템이 문자열인 PLAYER_ID를 왜 숫자로 나누는지 물어보는 과정 없이, 명확하게 숫자로 변환된 PLAYER_ID를 숫자로 나누게 되므로 성능이 향상됨
                        <img src="images/image1.png" width="30%">
                - CEIL(숫자)
                    - () 안에 있는 숫자랑 같거나 가장 가까이 큰 정수
                    - 예) CEIL(38.2) -> 39
                    - 예) CEIL(-38.2) -> 38
                    - 예) CEIL(38) -> 38
                - FLOOR(숫자)
                    - () 안에 있는 숫자랑 같거나 가장 가까이 작은 정수
                    - 예) FLOOR(-38.2) -> -39
                    - 예) FLOOR(38.2) -> 38
                    - 예) FLOOR(0) -> 0
                - ROUND(실수[, n])
                    - n개의 소숫점을 남겨서 반올림한 실수 출력
                    - n 없으면 소숫점 없이 반올림한 정수 출력
                    - 예) ROUND(12.2384, 2) -> 12.24
                - TRUNC(실수[, n])
                    - n개의 소숫점을 남겨서 나머지 값은 제거한 실수 출력
                    - n 없으면 소숫점 없이 나머지 값은 제거한 정수 출력
                    - 예) TRUNC(12.2384, 2) -> 12.23
                - EXP(숫자)
                    - 부동 소수점 인수 x(e x, 여기서 e는 가 2.17128128... 과 같음)의 지수 값을 연산
                    - 예) EXP(1) -> 2.17128128...
                    - 예) EXP(2) -> 7.389056...
                - POWER(숫자1, 숫자2)
                    - 숫자1의 숫자2 제곱
                    - 예) POWER(2, 3) -> 8 (2의 3승)
            - 변환 함수: 문자, 숫자, DATE형 값의 `데이터 타입 변환`
                - 예) `TO_CHAR()`, `TO_NUMBER()`, `TO_DATE()`
                - 종류: 명시적(explicit), 암시적(implicit)
                    - 명시적: 함수를 사용하여 명시적으로 데이터 타입 변환
                    - 암시적: 시스템이 자동으로 데이터 타입 변환
                        - 예) MOD(PLAYER_ID, 4) -> 문자열을 숫자로 변환
                        - 단점: 성능 저하 및 에러 발생의 가능성 존재
                        - 그러므로 암시적 데이터 변환은 왠만하면 사용 비추천
                - TO_CHAR(숫자|날짜[, FORMAT])
                    - 숫자나 날짜를 문자열로 변환
                    - SYSDATE는 시스템에 내제되어 있는 현재 날짜/시간을 반환해주는 변수
                    - 예) 
                        ```sql
                        SELECT TO_CHAR(SYSDATE, 'YYYY/MM/DD') AS 타입1,
                            TO_CHAR(SYSDATE, 'YYYY.MM.DD.HH24.MI.SS') AS 타입2,
                            TO_CHAR(SYSDATE) AS 타입3 
                        FROM DUAL;
                        ```
                - TO_NUMBER(문자열)
                    - 문자열을 숫자로 변환
                    - 암시적 변환:
                        ```sql
                        SELECT '1' + '1' AS 계산 
                        FROM DUAL;
                        ```
                    - 명시적 변환:
                        ```sql
                        SELECT TO_NUMBER('1') + TO_NUMBER('1') AS 계산
                        FROM DUAL
                        ```
                - TO_DATE(문자열[, FORMAT])
                    - 문자열을 날짜로 변환
                    - 예) 다음은 ERROR
                        ```sql
                        SELECT EXTRACT (YEAR FROM '20170123') AS 연도
                        FROM DUAL;
                        ```
                    - 예)
                        ```sql
                        SELECT EXTRACT (YEAR FROM TO_DATE('20170123', 'YYYY/MM/DD')) AS 연도
                        FROM DUAL;
                        ```
            - 날짜형 함수: DATE 타입의 변수 처리
                - 예) `SYSDATE()`, EXTRACT(), TO_NUMBER(TO_CHAR(d,'DD'|'MM'|'YY'))
                - SYSDATE
                    - 현재 날짜와 시각
                    - 예) 
                        ```sql
                        SELECT SYSDATE 
                        FROM DUAL;
                        ```
                        - 설명: DUAL이라는 테이블에 SYSDATE이라는 컬럼이 있는 것이 아니라, 문법을 맞추기 위하여 DUAL이라는 테이블에서 가져온다고 맞춘 것임
                    - 예) 
                        ```sql
                        SELECT TO_CHAR(SYSDATE, 'YYYY.MM.DD.HH24.MI.SS')
                        FROM DUAL;
                        ```
                - EXTRACT('YEAR'|'MONTH'|'DAY' FROM 날짜)
                    - 날짜 데이터에서 년/월/일 정보 추출
                    - 예)
                        ```sql
                        EXTRACT(YEAR FROM SYSDATE)
                        ```
                    - 예)
                        ```sql
                        SELECT EXTRACT(MONTH FROM BIRTH_DATE, 'DD') 
                        FROM DUAL;
                        ```
                - TRUNC(날짜, 'DD')
                    - 날짜 데이터에서 시/분/초를 잘라냄
                    - 예)
                        ```sql
                        SELECT TRUNC(SYSDATE, 'DD')
                        FROM DUAL;
                        ```
                    - 예) 
                        ```sql
                        SELECT TO_CHAR(TRUNC(SYSDATE, 'DD')), 'YYYY.MM.DD.HH24.MI.SS')
                        FROM DUAL;
                        ```
                - 연습문제: 태어난 날이 주어졌을 때, 그 날로부터 오늘까지 지난 날 수를 구하시오
                    - ```sql
                        SELECT TRUNC(SYSDATE - BIRTH_DATE[, 'DD']) AS DAY_PASSED
                        FROM PLAYER;
                        ```
            - 제어 함수: 논리값에 따른 값의 처리
                - 예) CASE, DECODE
                - CASE Expression
                    - 표현식이지만 함수의 성격을 갖고 있음
                    - IF ~ THEN ~ ELSE 논리 흐름
                    - 예) 
                        ```sql
                        SELECT PLAYER_NAME,
                            CASE WHEN HEIGHT > 180
                                THEN HEIGHT
                                ELSE 180
                            END AS NEW_HEIGHT
                        FROM PLAYER;
                        ```
                    |구분|예)|특징|
                    |--|--|--|
                    |Searched Case Expression|SELECT PLAYER_NAME,<br>&emsp;CASE<br>&emsp;&emsp;WHEN POSITION = 'GK' THEN '골키퍼'<br>&emsp;&emsp;WHEN POSITION = 'DF' THEN '수비수'<br>&emsp;&emsp;WHEN POSITION = 'MF' THEN '미드필더'<br>&emsp;&emsp;ELSE '그 외'<br>&emsp;END AS 포지션<br>FROM PLAYER;<br><br>SELECT PLAYER_NAME,<br>&emsp;CASE<br>&emsp;&emsp;WHEN HEIGHT > 185 THEN '장신'<br>&emsp;&emsp;ELSE (<br>&emsp;&emsp;&emsp;CASE<br>&emsp;&emsp;&emsp;&emsp;WHEN HEIGHT BETWEEN 175 AND 185 THEN '평균'<br>&emsp;&emsp;&emsp;&emsp;ELSE '단신'<br>&emsp;&emsp;&emsp;END)<br>&emsp;END AS 신장<br>FROM PLAYER;|- 표준 SQL<br>- 다양한 조건 사용 가능<br>- 표현식이 복잡함|
                    |Simple Case Expression|SELECT PLAYER_NAME,<br>&emsp;CASE POSITION<br>&emsp;&emsp;WHEN 'GK' THEN '골키퍼'<br>&emsp;&emsp;WHEN 'DF' THEN '수비수'<br>&emsp;&emsp;WHEN 'MF' THEN '미드필더'<br>&emsp;&emsp;ELSE '그 외'<br>&emsp;END AS 포지션<br>FROM PLAYER;|- 표준 SQL<br>- 동등(=) 비교에만 사용<br>- 표현식이 명료함|
                - DECODE
                    - Oracle에서만 사용
                    - DECODE(표현식, 기준값1, 출력값1[, 기준값2, 출력값2, ..., 디폴트값])
                    - 표현식의 값이 기준값1이면 값1을 출력, 기준값2이면 값2를 출력
                    - 기준값이 없으면 디폴트값을 출력
                    - 예) 
                        ```sql
                        SELECT PLAYER_NAME,
                            DECODE (POSITION,
                                'GK', '골키퍼',
                                'DF', '수비수',
                                'MF', '미드필더',
                                '그 외') AS POSITION
                        FROM PLAYER;
                        ```
            - NULL 관련 함수: NULL 처리
                - 예) `NVL()`, `NULLIF()`. COALESCE()
                - NULL이란:
                    - 비어있는 값
                    - 공백(space), 0과는 다른 의미
                    - NULL을 포함하는 모든 산술 연산의 결과는 NULL
                        - NULL+0, NULL-1, NULL*0, NULL/0 --> NULL
                    - NULL과 공집합(행 자체가 없는 경우)도 역시 다른 의미
                - NVL(표현식, 대체값)
                    - 표현식의 값이 NULL이면 대체값을, NULL이 아니면 표현식의 값을 반환
                    - 표현식의 값과 대체값의 데이터 타입이 같아야 함
                    - 얘) 
                    ```sql
                    SELECT PLAYER_NAME, POSITION, NVL(POSITION, '없음') AS 포지션
                    FROM PLAYER;
                    ```
                - NULLIF(표현식1, 표현식2)
                    - 두 식이 같으면 NULL을, 같지 않으면 표현식1의 값을 반환
                    - 특정 값을 NUlL로 반환할 때 사용
                    - 예)
                    ```sql
                    SELECT PLAYER_NAME, POSITION, NULLIF(POSITION, 'GK') AS 포지션
                    FROM PLAYER;
                    ```
                    - CASE 구문으로 표현 가능
                    ```sql
                    SELECT PLAYER_NAME, POSITION, 
                        CASE WHEN POSITION = 'GK' THEN NULL
                            ELSE POSITION
                        END AS 포지션
                    FROM PLAYER;
                    ```
                - COALESCE(표현식1, 표현식2, ...)
                    - 임의의 개수의 표현식에서 NULL이 아닌 최초의 표현식을 반환
                    - 모든 표현식이 NULL이라면 NULL을 반환
                    - 예) 
                    ```sql
                    SELECT E_PLAYER_NAME, NICKNAME, PLAYER_NAME, COALESCE(E_PLAYER_NAME, NICKNAME, PLAYER_NAME)
                    FROM PLAYER;
                    ```
        - 그룹 함수 (Group Function)
            - 집계 함수 (Aggregate Function) 포함
        - 윈도우 함수 (Window Function)

JOIN
- E.F.CODD의 연산자
    - SQL의 많은 기능이 RDB이론을 수립한 E.F.Codd 박사의 논문에서 소개됨
        - 4개의 일반집합연산자 + 4개의 순수관계연산자
        - 일반집합연사자의 SQL 구현
            1) UNION -> 현재 UNION/UNION ALL (공통집합의 중복 허용)
            2) INTERSECTION -> 현재 INTERSECT
            3) DIFFERENCE -> 현재 MINUS(Oracle)/EXCEPT(MS-SQL)
            4) PRODUCT -> 현재 CROSS JOIN

            <img src="images/image6.png" width="30%">
        - 순수관계연산자의 SQL 구현
            1) SELECT -> 현재 WHERE절
            2) PROJECT -> 현재 SELECT절
            3) JOIN -> 현재 매우 다양한 JOIN으로 구현됨
            4) DIVIDE -> 현재 사용되지 않음

            <img src="images/image7.png" width="30%">
- 정규화와 JOIN
    - 정규화
        - 이상현상(Anomaly) 발생을 피하기 위해 테이블을 분할하는 행위
        - 학계/실무에서 주로 3NF(3차 정규형) 사용
    - JOIN
        - 반정규화와 달리 JOIN은 스키마를 그대로 두고 (변형시키지 않고) 일시적인 테이블 연결을 통해 값 조회함
        - 데이터의 통합 조회를 위해 여러 테이블들을 연결
        - 실제 JOIN 연산은 두 개의 테이블에 대해서만 적용됨
        - 일반적인 경우 PK/FK의 연관에 의해 JOIN이 성립
            - 그 외에도 논리적인 값들의 연관성만으로 JOIN 성립 가능
- 예) 

    <img src="images/image8.png" width="30%">
- JOIN의 유형
    |분류|설명|
    |--|--|
    |Equi Join / Non-Equi Join|WHERE 테이블명.컬럼명 = 테이블명2.컬럼명2 <- 에서 =(equal) 연산을 사용하면 Equi, 미사용하면 Non-Equi|
    |암시적 조인 / 명시적 조인|JOIN이라는 명령어를 미사용시 암시적, 사용시 명시적|
    |Inner Join / Outer Join|추후 상세 설명|
    |Cross Join / Self Join|추후 상세 설명|
    
- EQUI JOIN (동등 조인)
    - 조인 조건으로 Equal(=) 연산 사용
    - 중복 컬럼의 경우, 걸럼명 앞에 테이블명을 붙여야 함
        - 중복되지 않는 컬럼도 컬럼명 앞에 테이블명을 붙이는 것 권장
        - 예) 선수명, 팀ID, 팀명 출력
            ```sql
            SELECT  EMP.ENAME, EMP.DEPTNO, DEPT.DNAME
            FROM    EMP, DEPT
            WHERE   EMP.DEPTNO = DEPT.DEPTNO
            ```
        - 테이블명이 긴 경우 ALIAS 사용
        - 예) 
            ```sql
            SELECT  E.ENAME, E.DEPTNO, D.DNAME
            FROM    EMP E, DEPT D
            WHERE   EMP.DEPTNO = DEPT.DEPTNO
            ```
        - FROM절에서 ALIAS 정의 후에는 WHERE/SELECT절에서 테이블명 사용 불가
        - 컬럼명이 한 테이블에만 있을시 ALIAS를 지정했더라도 꼭 앞에 테이블명ALIAS 안붙여도 됨
            ```sql
            SELECT  ENAME, E.DEPTNO, DNAME
            FROM    EMP E, DEPT D
            WHERE   EMP.DEPTNO = DEPT.DEPTNO
            ```
    - 셋 이상 테이블의 조인은 실제로는 두 테이블 간 조인이 연쇄적으로 일어남
        ```sql
        SELECT  P.PLAYER_NAME, P.POSITION, T.REGION_NAME, T.TEAM_NAME, S.STADIUM_NAME
        FROM    PLAYER P, TEAM T, STADIUM S
        WHERE   P.TEAM_ID = T.TEAM_ID AND T.STADIUM_ID = S.STADIUM_ID
        ```
        <img src="images/image9.png" width="30%">
- Non EQUI JOIN
    - 조인 조건으로 Equal(=) 이외의 연산 사용: BETWEEN, >, >=, <, <= 등
    - 예) 사원별 급여 등급 조회
    - 예)
    ```sql
    SELECT  E.ENAME 사원명, E.SAL 급여, S.GRADE 급여등급
    FROM    EMP E, SALGRADE S
    WHERE   E.SAL BETWEEN S.LOSAL AND S.HISAL;
    ```
    <img src="images/image10.png" width="10%">

- INNER JOIN (내부 조인)<br>&emsp;
    <img src="images/image11.png" width="10%">
    <img src="images/image12.png" width="30%">
    - 서로 대응되는 내용만 검색하는 조인
        - 조건절을 필수로 사용
    - 조인의 Default이므로 "INNER" 생략 가능
        - INNER JOIN = JOIN
    - 예) 암시적 조인
        ```sql
        SELECT  E.NAME, E.DEPTNO, E.SAL, D.DNAME
        FROM    EMP E, DEPT D
        WHERE   E.DEPTNO = D.DEPTNO AND E.SAL > 2000;
        ```
        - 문제: 조인 조건과 일반 조건이 혼용되어 가독성이 떨어짐 -> 명시적 조인 (=표준 조인)의 필요성
    - 명시적 조인에서는 조인 관련 조건은 ON 절에, 그 외의 조건은 WHERE 절에 기술함
    - 대부분의 DBMS는 명시적 조인을 표준으로 채택하지만, 기존의 암시적 조인도 허용
    - 예) 명시적 조인 (표준)
        ```sql
        SELECT  E.NAME, E.DEPTNO, E.SAL, D.DNAME
        FROM    EMP E INNER JOIN DEPT D
        ON      E.DEPTNO = D.DEPTNO 
        WHERE   E.SAL > 2000;
        ```
        - 위의 INNER JOin에서 INNER는 생략 가능
    - NATURAL JOIN
        - INNER JOIN의 특수한 경우
            - NATURAL INNER JOIN = NATURAL JOIN
        - 두 테이블 간 동일한 이름을 갖는 모든 컬럼들에 대해 EQUI JOIN 수행
            - 컬럼 간 `데이터 타입도 동일`해야 함
            - 별도의 조인 컬럼 및 조건 `지정 불가`
        - 조인의 대상이 되는 컬럼에는 `접두사(테이블명 또는 ALIAS)를 사용 불가`
        - 예) 
            ```sql
            SELECT  EMPNO, ENAME, DEPTNO, DNAME
            FROM    EMP NATURAL JOIN DEPT;
            ```
        - 위를 INNER JOIN 질의로 작성시
            ```sql
            SELECT  E.EMPNO, E.ENAME, D.DEPTNO, D.DNAME
            FROM    EMP E INNER JOIN DEPT D
            ON      E.EMPNO = D.EMPNO
            ON      E.ENAME = D.ENAME
            ON      E.DEPTNO = D.DEPTNO
            ON      E.DNAME = D.DDNAME
            ```
        - NATURAL 과 INNER 조인의 차이점
            - SELECT * FROM EMP NATURAL JOIN DEPT;
            - SELECT * FROM EMP INNER JOIN DEPT ON EMP.DEPTNO = DEPT.DEPTNO;
            - 위와 같은 경우, EMP와 DEPT의 테이블에서 서로 같은 컬럼이 DEPTNO만 있는 경우,
                - NATURAL JOIN은 DEPTNO의 컬럼을 하나만 출력하지만 INNER JOIN은 DEPTNO, DEPTNO_1 이렇게 2개의 컬럼을 출력해준다<br>
            <img src="images/image13.png" width="30%">
- JOIN 조건절
    - ON 조건절
        - 암시적 JOIN
            - JOIN이라는 명령어 미사용
            - 모든 조건을 WHERE절에 기술
        - 명시적 JOIN
            - JOIN 기준 조건은 ON절에 기술 (ON 절의 괄호는 생략 가능)
            - JOIN과 무관한 일반 조건은 WHERE절에 기술
        - 예) 이름에 'S'를 포함하는 사원의 사원이름, 부서코드, 부서명 출력
            ```sql
            SELECT  E.EMPNO, E.ENAME, E.DEPTNO, D.DNAME
            FROM    EMP E JOIN DEPT D
            ON      (E.DEPTNO = D.DEPTNO)
            WHERE   E.ENAME LIKE '%S%';
            ```
    - USING 조건절
        - ON절의 '=' 연산자 대신 USING절 사용 가능
            - ON절에서는 괄호 생략 가능, USING서는 괄호 생략 불가
        - 접두사(테이블명 또는 ALIAS) 사용 불가
        
        |ON 조건절|USING 조건절|
        |--|--|
        |SELECT&emsp;E.ENAME, E.DEPTNO, D.DNAME<br>FROM&emsp;&emsp;EMP E JOIN DEPT D<br>ON&emsp;&emsp;&emsp;(E.DEPTNO = D.DEPTNO)|SELECT&emsp;ENAME, DEPTNO, DNAME<br>FROM&emsp;&emsp;EMP JOIN DEPT<br>USING&emsp;&emsp;(DEPTNO)|
- WHERE, ON, USING절 조건 기술 비교<br>
    <img src="images/image14.png" width="30%"><br>
- OUTER JOIN (외부 조인)
    - 서로 대응되지 않는 행도 출력하는 조인
    - 조건절을 필수로 사용
    - 성능 저하의 원인이 될 수 있으므로 필요한 경우만 사용<br>
    <img src="images/image15.png" width="30%"><br>
    - LEFT OUTER JOIN
        - 왼쪽 테이블의 데이터를 모두 읽은 후, 오른쪽 테이블에서 JOIN 데이터를 가져옴
        - 오른쪽 테이블이 JOIN 조건에 해당되지 않는 경우, 해당 컬럼은 NULL로 채움<br>
        <img src="images/image16.png" width="30%">
    - RIGHT OUTER JOIN
        - 오른쪽 테이블의 데이터를 모두 읽은 후, 왼쪽 테이블에서 JOIN 데이터를 가져옴
        - 왼쪽 테이블이 JOIN 조건에 해당되지 않는 경우, 해당 컬럼은 NULL로 채움<br>
        <img src="images/image17.png" width="30%">
    - FULL OUTER JOIN
        - 양쪽 테이블의 데이터를 모두 읽은 후, 상대 테이블에서 JOIN 데이터를 가져옴
        - JOIN 조건에 해당되지 않는 경우, 해당 컬럼은 NULL로 채움<br>
        <img src="images/image18.png" width="30%">    
        - RIGHT OUTER JOIN과 LEFT OUTER JOIN의 합집합과 동일 (중복 제거 후)
            - 즉, UNION ALL이 아닌 UNION과 동일
            <img src="images/image19.png" width="30%">
        - UNION vs UNION ALL
            - UNION은 FULL OUTER JOIN (RIGHT OUTER JOIN과 LEFT OUTER JOIN의 합집한)과 동일
            - UNION ALL은 RIGHT OUTER JOIN 밑에 LEFT OUTER JOIN을 붙여놓은 것과 동일
- CROSS JOIN (교차 조인)
    - 두 테이블의 곱집합(Cartesian Product)을 출력하는 조인
    - 별도의 조인 조건 없음<br>
    <img src="images/image20.png" width="30%">
- SELF JOIN (셀프 조인)
    - 동일 테이블 사이의 조인
        - FROM 절에 동일 테이블이 두 번 이상 나타남
    - 테이블 식별을 위해 반드시 별칭(alias)을 사용해야 함
        - 동일한 테이블을 개념적으로 서로 다른 두 개의 테이블로 사용함
        - 예) 
            ```sql
            FROM EMP E INNER JOIN EMP M
            ```
            <img src="images/image21.png" width="30%">

계층형 질의
- 계층형 데이터
    - 동일 테이블에 계층적으로 상위와 하위 데이터가 포함된 데이터
    - 엔터티가 순환관계 모델로 설계된 경우 발생
- 계층형 질의 (Hierarchical Query)를 통해 접근 가능<br>
    <img src="images/image22.png" width = "30%">
    - 계층형 질의의 방향<br>
        <img src="images/image23.png" width="30%">
    - 계층형 질의의 구조
        - START WITH - 시작 조건 지정
            - 예) 
                ```sql
                START WITH MGR IS NULL
                ```
                ```sql
                START WITH EMPNO='D'
                ```
        - CONNECT BY - 다음에 전개될 방향 지정<br>
            <img src="images/image24.png" width="10%">
            - (순방향) PRIOR 자식 = 부모
                - 예) 
                    ```sql
                    CONNECT BY PRIOR EMPNO = MGR
                    ```
            - (역방향) PRIOR 부모 = 자식
                - 예) 
                    ```sql
                    CONNECT BY PRIOR MGR = EMPNO
                    ```

            - 순방향 계층형 질의 예)<br>
                <img src="images/image25.png" width="30%">
                <img src="images/image26.png" width="30%">
    - PSEUDO COLUMN
        - LEVEL: 시작 노드 = 1, Leaf까지 1씩 증가
        - CONNECT_BY_ISLEAF = 해당 노드의 후속 노드가 없으면 1; 즉, Leaf이면 1, 그렇지 않으면 0
        - CONNECTED_BY_ROOT: 시작 노드의 해당 컬럼 표시
        - SYS_CONNECTED_BY_PATH: 시작 노드부터 현재 노드까지 경로 표시

집합 연산자
- 개요
    - 여러 질의(Select 문) 결과를 하나로 결합하기 위해 사용
    - 집합 연산의 대상이 되는 두 질의는:    
        - SELECT 절의 컬럼 수가 동일해야 하고
        - SELECT 절의 동일 위치에 존재하는 컬럼의 데이터 타입이 상호 호환 가능해야 함
            - 반드시 동일한 데이터 타입일 필요는 없음
            
    |집합 연산자|연산자 의미|
    |:-:|--|
    |UNION|여러 SQL문의 결과에 대한 합집합<br>(중복된 행은 제거한 후 하나의 행만 출력)|
    |UNION ALL|여러 SQL문의 결과에 대한 합집합<br>(중복된 행도 삭제하지 않고 모두 출력-> 속도가 빠르므로 우선 고려)|
    |INTERSECT|여러 SQL문의 결과에 대한 교집합<br>(중복된 행은 제거한 후 하나의 행만 출력)|
    |MINUS (Oracle)<br><br>EXCEPT (MS-SQL)|앞 SQL문의 결과에서 뒤 SQL문의 결과를 뺀 차집합<br>(중복된 행은 제거한 후 하나의 행만 출력)|

    <br><img src="images/image27.png" width="30%">
- 집합 연산 질의 예)
    - 집합 연산은 둘 이상의 SELECT문을 결합하는 것
    - ORDER BY는 집합 연산을 적용한 최종 결과에 대한 정렬
        - 맨 마지막 줄에 한 번만 기술함
- UNION ALL
    ```sql
    SELECT  PLAYER_NAME, BACK_NO, TEAM_ID
    FROM    PLAYER
    WHERE   TEAM_ID = 'K04'
    UNION ALL
    SELECT  PLAYER_NAME, BACK_NO, TEAM_ID
    FROM    PLAYER
    WHERE   TEAM_ID = 'K06'
    ORDER BY PLAYER_NAME;
    ```
    - 이질적 성격의 데이터를 한꺼번에 출력하는 연산도 가능
        ```sql
        SELECT  'T' 구분코드, PLAYER_NAME, TEAM_ID
        FROM    PLAYER
        WHERE   TEAM_ID = 'K06'
        UNION ALL
        SELECT  'P' 구분코드, PLAYER_NAME, POSITION
        FROM    PLAYER
        WHERE   POSITION = 'GK'
        ORDER BY 구분코드, TEAM_ID;
        ```
        - 'P'와 'T'는 상수값
    - 출력 컬럼의 컬럼명은 첫 SELECT문의 컬럼명이 적용됨
        - ORDER BY 구분코드, POSITION -> ERROR!!
        <img src="images/image28.png" width="10%">
- INTERSEECT
    ```sql
    SELECT  TEAM_ID 팀코드, PLAYER_NAME 선수명, POSITION 포지션
    FROM    PLAYER
    WHERE   TEAM_ID = 'K06'
    INTERSECT
    SELECT  TEAM_ID 팀코드, PLAYER_NAME 선수명, POSITION 포지션
    FROM    PLAYER
    WHERE   POSITION = 'GK'

    는 다음과 같다

    SELECT  TEAM_ID 팀코드, PLAYER_NAME 선수명, POSITION 포지션
    FROM    PLAYER
    WHERE   TEAM_ID = 'K06' AND POSITION = 'GK'
    ```
    - INTERSECT 연산자는 IN 서브쿼리, EXISTS 서브쿼리로도 표현 가능
        - -> 서브쿼리에서 설명
- MINUS
    ```sql
    SELECT  TEAM_ID 팀코드, PLAYER_NAME 선수명, POSITION 포지션
    FROM    PLAYER
    WHERE   TEAM_ID = 'K06'
    MINUS
    SELECT  TEAM_ID 팀코드, PLAYER_NAME 선수명, POSITION 포지션
    FROM    PLAYER
    WHERE   POSITION = 'MF';

    는 다음과 같다

    SELECT  TEAM_ID 팀코드, PLAYER_NAME 선수명, POSITION 포지션
    FROM    PLAYER
    WHERE   TEAM_ID = 'K06' AND POSITION <> 'MF';

    는 다음과 같다

    SELECT  TEAM_ID 팀코드, PLAYER_NAME 선수명, POSITION 포지션
    FROM    PLAYER
    WHERE   TEAM_ID = 'K06'
    AND     PLAYER_ID NOT IN (SELECT PLAYER_ID FROM PLAYER WHERE POSITION = 'MF');
    ```
- 집합 연산과 ALIAS

    <img src="images/image29.png" width="50%">